# parseia-json
Esse script transforma os dados do CSV, já filtrado, manualmente checado e com apenas os discursos que mencionam a ditadura, e transforma em um formato .json para gerar a visualização de dados.

#### Importar pacotes

In [1]:
import json
import pandas as pd
import pprint as pp
import re

#### Formatar dados ao longo do tempo

In [2]:
def resample_df(df):
    
    '''
    Essa função recebe o dataframe que veio de 'buscador-da-ditadura.ipynb',
    mas com duas colunas extras ("HIGHLIGHT" e "IS_MATCH") que indicam,
    respectivamente, o trecho para ser destacado no interativo e se a
    entrada realmente menciona a ditadura militar.
    '''
    
    # Filtra para manter os anos que temos inteiros
    df = df['2001-01-01':'2018-12-31']
    
    # Baseado em: stackoverflow.com/questions/45908146/

    # Cria um índice no intervalo desejado
    t_index = pd.DatetimeIndex(start='2001-01-01', end='2018-12-31', freq='M')

    # Faz o resample e agregação
    df = df.resample('M').agg({
        # Primeiro, contamos quantos discursos foram registrados no mês
        "FILE":'count', 
        # Depois, quantos deles mecionaram a ditadura.
        # Soma funciona porque, quando há uma menção, o valor é 1. Quando não é, o valor é zero
        "IS_MATCH":'sum'
    })

    # Aplica no novo índice
    df = df.reindex(t_index)

    # Reseta índice
    df = df.reset_index()

    # Renomeia colunas
    df = df.rename( columns = {
        'index' : "SESSION_DATE",
        'FILE'  : 'ALL_MENTIONS'
    })

    # Preenche os NaNs com zero
    df = df.fillna(0)
    
    return df

#### Recuperar urls dos dados

Para permitirmos que o leitor acesse a íntegra dos discursos destacados, vamos recuperar o link de cada discurso, que está disponível nos metadados do diretório `table`.

In [3]:
def get_urls(row, metadata):
    
    '''
    Função para ser rodada via df.apply, que
    usa o nome de um arquivo para recuperar
    o link para o conteúdo original.
    '''
    
    i = re.search('/(\d+)-.*', row.FILE)[1]
    i = int(i)

    try:
        url = metadata.loc[i, 'Discurso']
    except:
        url = 'PLACEHOLDER'
    
    return pd.Series({
        'URL' : url
    })
    
    

#### Parsear json

In [4]:
def parse_json_chart(df, df_t):

    '''
    PARÂMETROS:
    
    df -> O dataframe original, com todas as declarações, já
    verificado manualmente e com as colunas 'IS_MATCH' e
    'HIGHLIGHT' presentes.

    df_t -> O dataframe que já passou pelo resample,
    que é feito na função resample_df(()
    '''
    
    array = [ ]
    
    for index, row in df_t.iterrows():

        # Seleciona as votações que ocorreram no dia
        this_year  = row.SESSION_DATE.year
        this_month = row.SESSION_DATE.month
        sliced_df  = df[ f"{this_year}-{this_month}" ]

        # Filta para manter apenas aquelas em que houve uso de palavra-chave
        sliced_df = sliced_df[ sliced_df.IS_MATCH == 1 ]

        # Reordena de mais recente para mais tardia
        sliced_df = sliced_df.sort_index().reset_index()

        
        obj = { }

        obj["year"]       = int(row.SESSION_DATE.year)
        obj["month"]      = int(row.SESSION_DATE.month)
        obj["total"]      = int(row.ALL_MENTIONS)
        obj["highlights"] = int(row.IS_MATCH)
        obj["speeches"]   = [ ]
        
        for i in range(obj["highlights"]):
            inner_obj = { }
            inner_obj["excerpt"] = sliced_df.loc[i, 'HIGHLIGHT']
            inner_obj["session"] = sliced_df.loc[i, 'SESSION_TYPE']
            inner_obj["date"]    = str( sliced_df.loc[i, 'SESSION_DATE'] )[:10]
            inner_obj["link"]    = str( sliced_df.loc[i, 'URL'])

            obj["speeches"].append(inner_obj)

        array.append(obj)
        
    return array

#### Executar operações

In [5]:
def run_script_chart():
    
    path = "../data/csvs/DISCURSOS_CHECADOS_MANUALMENTE.csv"
    df = pd.read_csv(path)
    
     # Remove dados de discursos em comissão
    df = df[ df.SESSION_TYPE != 'COMISSÃO ESPECIAL (FORA DO PLENÁRIO)' ]
    
    # Pega a URL correta
    metadata = pd.read_csv("../data/tables/jair-bolsonaro-plenario-metadata.csv")
    df["URL"] = df.apply( get_urls, args = (metadata,), axis = 1 )
    
    # Passa a data do pronunciamento para o index
    df["SESSION_DATE"] = pd.to_datetime(df.SESSION_DATE)
    df = df.set_index("SESSION_DATE")
    
    # Faz o resample temporal
    df_t = resample_df(df)

    array = parse_json_chart(df, df_t)
        
    with open('../data/jsons/data.json', 'w') as fp:
        json.dump(array, fp, indent = 2)
        
    pp.pprint(array)

A visualização também exige um segundo JSON, com o texto dos destaques, que será gerado abaixo.

In [6]:
def parse_json_highlights():
    
    df = pd.read_csv("../data/csvs/sneakpeek.csv")
    df["DATE"] = pd.to_datetime(df.DATE)
    
    array = [ ]
    
    for index, row in df.iterrows():
        
        obj = { }
        
        obj["year"]   = row.DATE.year
        obj["month"]  = row.DATE.month
        obj["speech"] = row.SPEECH_INDEX
        obj["text"]   = row.TEXT
        
        array.append(obj)
        
    with open('../data/jsons/sneakpeeks.json', 'w') as fp:
        json.dump(array, fp, indent = 2)
    
    pp.pprint(array)

In [7]:
parse_json_highlights()

[{'month': 8,
  'speech': 0,
  'text': 'Está claro que a âncora do real é o arrocho salarial e a humilhação '
          'dos menos favorecidos. Essa é uma atitude covarde. Não poderíamos '
          'esperar outra coisa do cidadão Fernando Henrique Cardoso, que, '
          'infelizmente, o regime militar não mandou para o espaço.',
  'year': 2001},
 {'month': 3,
  'speech': 0,
  'text': 'O regime militar, tão condenado, deixou vivo Aloysio Nunes '
          'Ferreira, portanto não era tão violento como dizem. [...] Por falar '
          'em violência, não existe hoje classe que esteja sendo mais '
          'violentada do que a dos militares das Forças Armadas.',
  'year': 2002},
 {'month': 7,
  'speech': 1,
  'text': 'Ouvi há pouco no corredor as pessoas pedindo a volta da ditadura e '
          'das cassações para varrer da Pátria esses [petistas] traidores, '
          'mandando-os para o jângal, para o valão. É isso que eles merecem: '
          'ditadura já!',
  'year': 2003},
 {